In [1]:
import pandas as pd
from collections import defaultdict
from quickUmls.client import get_quickumls_client
import pickle
from os import mkdir
from os.path import join
from os import listdir
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re

In [2]:
pattern = "(?:^(?:never|no|nothing|nowhere|noone|none|not|havent|hasnt|hadnt|cant|couldnt|shouldnt|wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint)$)|n't"
endSymbolsTillNegation = [',', '.', ':', ';', '!', '?']
#  ‘,’,
# ‘.’, ‘:’, ‘;’, ‘!’, ‘?’.

In [3]:
stopWords = stopwords.words('english')
print(stopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
removeSymbolsList = ['∆', '(', ')', ',', '.', 'β', 'α', "'s'", '$', '``', "''", "'s", ':', ';', '/', '\\', '+']

In [5]:
matcher = get_quickumls_client()

In [6]:
lemmatizer = WordNetLemmatizer()

In [7]:
ps = PorterStemmer()

In [8]:
input_query_path = "../data/githubAnnotatedDataAndIncExcAppended"
output_path = "../data/queryWordsExactWordsFieldWiseFrequency"

In [9]:
# mkdir(output_path)

In [10]:
files = sorted(listdir(input_query_path))
files

['Dietary Therapy Epilepsies_page_rank.csv',
 'Early Parkinson disease treatment_page_rank.csv',
 'HIV infection Treatment naive_page_rank.csv',
 'HIV infection seronegativity_page_rank.csv',
 'Hypercholesterolemia safe treatments_page_rank.csv',
 'Nonvalvular atrial fibrillation_page_rank.csv',
 'Outcomes of cerebrovascular accident_page_rank.csv',
 'Treating Anemia, Iron-Deficiency in CKD patients_page_rank_15.csv',
 'already having Celiac Disease_page_rank.csv',
 'antiretroviral therapy first time_pageRank.csv',
 'constipation safe treatments_page_rank.csv',
 'dietary approaches for obesity treatment_page_rank.csv',
 'haemorrhage cure_page_rank.csv',
 'hypertension safe treatments_page_rank.csv',
 'low back pain therapy workout_pageRank.csv',
 'malnutrition in young children_page_rank.csv',
 'managing constipation in children_pageRank.csv',
 'out of hospital cardiac arrest_page_rank.csv',
 'postoperative delirium_pageRank.csv',
 'recommended anti-platelet doses for treating Coronary

In [11]:
def getListOfWordsForWhichUMLSConceptdidntGen(query):
    final_query = query
    concepts = matcher.match(query, best_match=True, ignore_syntax=False)
    conceptOfQuery = [elem[0][u'cui'] for elem in concepts]
#     print(conceptOfQuery)
    for concept in concepts:
        toRemove = ""
        toRemove += query[concept[0]['start']:concept[0]['end']]
        final_query = final_query.replace(toRemove, "")
        final_query = final_query.replace("  ", " ")
#     print(final_query)
    
    final_query = final_query.split()
    final_query = [ps.stem(word) for word in final_query]
#     final_query = [lemmatizer.lemmatize(word) for word in final_query]
    final_query = [word for word in final_query if word not in stopWords]
    final_query = [word.lower() for word in final_query]
    return final_query

In [12]:
## Brieftitle
def briefTitle(df, queryWordsToSearch):
    matchedCountList = []
    df['brief_title'] = df['brief_title'].fillna("")
    for title in df['brief_title']:
        matched = 0
#         print(title)
        tokenizedWords = word_tokenize(title)
        newL = []
        for word in tokenizedWords:
            if '-' in word:
                newL += word.split('-')
                
            else:
                newL.append(word)
        
        tokenizedWords = newL
        tokenizedWords = [ps.stem(word) for word in tokenizedWords]
        tokenizedWords = [word for word in tokenizedWords if word not in stopWords]
        tokenizedWords = [word for word in tokenizedWords if word not in removeSymbolsList]
        tokenizedWords = [word.lower() for word in tokenizedWords]
#         print(tokenizedWords)
        
        for queryWord in queryWordsToSearch:
            if queryWord in tokenizedWords:
                matched += 1
        matchedCountList.append(matched)
#     print(matchedCountList)
    return matchedCountList

In [13]:
## Official Title
def officialTitle(df, queryWordsToSearch):
    matchedCountList = []
    df['official_title'] = df['official_title'].fillna("")
    for title in df['official_title']:
        matched = 0
#         print(title)
        tokenizedWords = word_tokenize(title)
        newL = []
        for word in tokenizedWords:
            if '-' in word:
                newL += word.split('-')
                
            else:
                newL.append(word)
        
        tokenizedWords = newL
        tokenizedWords = [ps.stem(word) for word in tokenizedWords]
        tokenizedWords = [word for word in tokenizedWords if word not in stopWords]
        tokenizedWords = [word for word in tokenizedWords if word not in removeSymbolsList]
        tokenizedWords = [word.lower() for word in tokenizedWords]
#         print(tokenizedWords)
        
        for queryWord in queryWordsToSearch:
            if queryWord in tokenizedWords:
                matched += 1
        matchedCountList.append(matched)
#     print(matchedCountList)
    return matchedCountList

In [14]:
## Conditions List
def conditionsList(df, queryWordsToSearch):
    matchedCountList = []
    df['conditions_list'] = df['conditions_list'].fillna("")
    for title in df['conditions_list']:
        matched = 0
#         print(title)
        tokenizedWords = title.split(';')
        newL = []
        for word in tokenizedWords:
            newL += word_tokenize(word)
        tokenizedWords = newL
        tokenizedWords = [ps.stem(word) for word in tokenizedWords]
        tokenizedWords = [word for word in tokenizedWords if word not in stopWords]
        tokenizedWords = [word for word in tokenizedWords if word not in removeSymbolsList]
        tokenizedWords = [word.lower() for word in tokenizedWords]
#         print(tokenizedWords)
        
        for queryWord in queryWordsToSearch:
            if queryWord in tokenizedWords:
                matched += 1
        matchedCountList.append(matched)
#     print(matchedCountList)
    return matchedCountList

In [15]:
## Keywords List
def keywordsList(df, queryWordsToSearch):
    matchedCountList = []
    df['keywords_list'] = df['keywords_list'].fillna("")
    for title in df['keywords_list']:
        matched = 0
#         print(title)
        tokenizedWords = title.split(';')
        newL = []
        for word in tokenizedWords:
            newL += word_tokenize(word)
        tokenizedWords = newL
        tokenizedWords = [ps.stem(word) for word in tokenizedWords]
        tokenizedWords = [word for word in tokenizedWords if word not in stopWords]
        tokenizedWords = [word for word in tokenizedWords if word not in removeSymbolsList]
        tokenizedWords = [word.lower() for word in tokenizedWords]
#         print(tokenizedWords)
        
        for queryWord in queryWordsToSearch:
            if queryWord in tokenizedWords:
                matched += 1
        matchedCountList.append(matched)
#     print(matchedCountList)
    return matchedCountList

In [16]:
## Brief Summary
def briefSummary(df, queryWordsToSearch):
    row = 2
    matchedCountList = []
    df['description'] = df['description'].fillna("")
    for title in df['description']:
        matched = 0
#         print(title)
        tokenizedWords = word_tokenize(title)
        newL = []
        for word in tokenizedWords:
            if '-' in word:
                newL += word.split('-')
                
            else:
                newL.append(word)
        
        tokenizedWords = newL
        tokenizedWords = [ps.stem(word) for word in tokenizedWords]
        tokenizedWords = [word for word in tokenizedWords if word not in stopWords]
        tokenizedWords = [word for word in tokenizedWords if word not in removeSymbolsList]
        tokenizedWords = [word.lower() for word in tokenizedWords]
#         print(row)
#         print(tokenizedWords)
#         print()
        
        for queryWord in queryWordsToSearch:
            if queryWord in tokenizedWords:
                matched += 1
        matchedCountList.append(matched)
        row += 1
#     print(matchedCountList)
    return matchedCountList

In [17]:
def processEachInclusionRow(inc_sentences):
    matched = 0
    bigWordListForAllSentences = []

    for sentence in inc_sentences.split(';;;;;;;;;;'):
        sentence = sentence.replace("             ", " ")
        sentence = word_tokenize(sentence)
#             print(sentence)
        bigWordListForAllSentences += sentence
        bigWordListForAllSentences.append(".")
    
    bigWordListForAllSentences = [word.lower() for word in bigWordListForAllSentences]
    
    bigWordNegationFlagList = [0]*len(bigWordListForAllSentences)
    
    
    wordIndex = 0
    while wordIndex < len(bigWordNegationFlagList):
        word = bigWordListForAllSentences[wordIndex]
        regMatch = re.search(pattern, word)
        if regMatch:
#             print("TRUE", word)
            while wordIndex < len(bigWordNegationFlagList) and bigWordListForAllSentences[wordIndex] not in endSymbolsTillNegation:
                bigWordNegationFlagList[wordIndex] = 1
                wordIndex += 1
        wordIndex += 1
    
    
    bigWordListForAllSentences = [ps.stem(word) for word in bigWordListForAllSentences]
#     print("BigList:", bigWordListForAllSentences)
#     print(bigWordNegationFlagList)
    
    return bigWordListForAllSentences, bigWordNegationFlagList

In [18]:
def findWordInInclusionContext(queryWordsToSearch, bigWordListForAllSentences, bigWordNegationFlagList):
    matchedCountPos = 0
    matchedCountNeg = 0
    for query_word in queryWordsToSearch:
        for sent_word_neg_flag_tup in zip(bigWordListForAllSentences, bigWordNegationFlagList):
            sent_word = sent_word_neg_flag_tup[0]
            sent_word_context_flag = sent_word_neg_flag_tup[1]
            
            if query_word == sent_word:
                if sent_word_context_flag==0:
                    matchedCountPos += 1
                else:
                    matchedCountNeg += 1
#                 print("Matched:", "Query:Word", query_word, "SentenceWord:", sent_word, "Context:", sent_word_context_flag)
    return matchedCountPos, matchedCountNeg

In [19]:
def inclusionCriteria(df, queryWordsToSearch):
    matchedCountListPos = []
    matchedCountListNeg = []
    
    df['inclusion'] = df['inclusion'].fillna("")
    
    for inc_sentences in df['inclusion']:
        bigWordListForAllSentences, bigWordNegationFlagList = processEachInclusionRow(inc_sentences)
        matchedCountPos, matchedCountNeg = findWordInInclusionContext(queryWordsToSearch, bigWordListForAllSentences, bigWordNegationFlagList)
        
        matchedCountListPos.append(matchedCountPos)
        matchedCountListNeg.append(matchedCountNeg)
        
    return matchedCountListPos, matchedCountListNeg

In [20]:
def processEachExclusionRow(exc_sentences):
    matched = 0
    bigWordListForAllSentences = []

    for sentence in exc_sentences.split(';;;;;;;;;;'):
        sentence = sentence.replace("             ", " ")
        sentence = word_tokenize(sentence)
#             print(sentence)
        bigWordListForAllSentences += sentence
        bigWordListForAllSentences.append(".")
    
    bigWordListForAllSentences = [word.lower() for word in bigWordListForAllSentences]
    
    bigWordNegationFlagList = [0]*len(bigWordListForAllSentences)
    
    
    wordIndex = 0
    while wordIndex < len(bigWordNegationFlagList):
        word = bigWordListForAllSentences[wordIndex]
        regMatch = re.search(pattern, word)
        if regMatch:
#             print("TRUE", word)
            while wordIndex < len(bigWordNegationFlagList) and bigWordListForAllSentences[wordIndex] not in endSymbolsTillNegation:
                bigWordNegationFlagList[wordIndex] = 1
                wordIndex += 1
        wordIndex += 1
    
    
    bigWordListForAllSentences = [ps.stem(word) for word in bigWordListForAllSentences]
#     print("BigList:", bigWordListForAllSentences)
#     print(bigWordNegationFlagList)
    
    return bigWordListForAllSentences, bigWordNegationFlagList

In [21]:
def findWordInExclusionContext(queryWordsToSearch, bigWordListForAllSentences, bigWordNegationFlagList):
    matchedCountPos = 0
    matchedCountNeg = 0
    
    for query_word in queryWordsToSearch:
        for sent_word_neg_flag_tup in zip(bigWordListForAllSentences, bigWordNegationFlagList):
            sent_word = sent_word_neg_flag_tup[0]
            sent_word_context_flag = sent_word_neg_flag_tup[1]
            
            if query_word == sent_word:
                if sent_word_context_flag == 0:
                    matchedCountPos += 1
                else:
                    matchedCountNeg += 1
#                 print("Matched:", "Query:Word", query_word, "SentenceWord:", sent_word, "Context:", sent_word_context_flag)
    return matchedCountPos, matchedCountNeg

In [22]:
def exclusionCriteria(df, queryWordsToSearch):
    matchedCountListPos = []
    matchedCountListNeg = []
    
    df['exclusion'] = df['exclusion'].fillna("")
    for inc_sentences in df['exclusion']:
        bigWordListForAllSentences, bigWordNegationFlagList = processEachInclusionRow(inc_sentences)
        matchedCountPos,  matchedCountNeg = findWordInExclusionContext(queryWordsToSearch, bigWordListForAllSentences, bigWordNegationFlagList)
        matchedCountListPos.append(matchedCountPos)
        matchedCountListNeg.append(matchedCountNeg)
    return matchedCountListPos, matchedCountListNeg

In [23]:
for file in files:
    query = file.split("_")[0]
    print(query)
    queryWordsToSearch = getListOfWordsForWhichUMLSConceptdidntGen(query)
    print(queryWordsToSearch)
    df = pd.read_csv(join(input_query_path, file))
#     print(df.columns)
    
    matchList = briefTitle(df, queryWordsToSearch)
    df['brief_title_matched_count'] = matchList
    
    matchList = officialTitle(df, queryWordsToSearch)
    df['official_title_matched_count'] = matchList
    
    matchList = conditionsList(df, queryWordsToSearch)
    df['conditions_list_matched_count'] = matchList
    
    matchList = keywordsList(df, queryWordsToSearch)
    df['keywords_list_matched_count'] = matchList
    
#     print(keywordsList(df, queryWordsToSearch))
    matchList = briefSummary(df, queryWordsToSearch)
    df['brief_summary_matched_count'] = matchList
    
    matchedCountListPos, matchedCountListNeg = inclusionCriteria(df, queryWordsToSearch)
    df['inclusion_pos'] = matchedCountListPos
    df['inclusion_neg'] = matchedCountListNeg
    
    matchedCountListPos, matchedCountListNeg = exclusionCriteria(df, queryWordsToSearch)
    df['exclusion_pos'] = matchedCountListPos
    df['exclusion_neg'] = matchedCountListNeg
    
    df.to_csv(join(output_path, file), index=False)

Dietary Therapy Epilepsies
['dietari']
Early Parkinson disease treatment
['earli']
HIV infection Treatment naive
['naiv']
HIV infection seronegativity
['seroneg']
Hypercholesterolemia safe treatments
['safe']
Nonvalvular atrial fibrillation
['nonvalvular']
Outcomes of cerebrovascular accident
['outcom']
Treating Anemia, Iron-Deficiency in CKD patients
['treat', 'ckd', 'patient']
already having Celiac Disease
['alreadi']
antiretroviral therapy first time
['first', 'time']
constipation safe treatments
['safe']
dietary approaches for obesity treatment
['dietari', 'approach']
haemorrhage cure
['cure']
hypertension safe treatments
['safe']
low back pain therapy workout
['workout']
malnutrition in young children
['young', 'children']
managing constipation in children
['manag', 'children']
out of hospital cardiac arrest
['hospit']
postoperative delirium
['postop']
recommended anti-platelet doses for treating Coronary artery disease
['recommend', 'anti-platelet', 'dose', 'treat']
safe treatmen